In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import datetime as dt
import torch.nn as nn
from torchdiffeq import odeint_adjoint as odeint
import csv
from pathlib import Path
from scipy import signal
from ipywidgets import interact
import matplotlib.pyplot as plt

import torch.nn.functional as F


path = os.path.join(os.getcwd(),"datasets","Longer")

def files(path, ext=None):
    for (dirpath, dirnames, filenames) in os.walk(path):
        for file in filenames:
            if os.path.isfile(os.path.join(dirpath, file)):
                f_dta = None
                for file_dta in filenames:
                    if file.split(".")[0] in file_dta and file != file_dta:
                        f_dta = file_dta
                
                if ext is None:
                    yield dirpath, file, f_dta
                elif ext in file.split(".")[-1]:
                    yield dirpath, file, f_dta


file_paths = []
for file in files(path, "csv"):
    file_paths.append(file)

In [ ]:

def read_from_csv_with_datetime(file_path):
    with open(file_path) as csv_file:
        reader = csv.reader(csv_file, delimiter=',')
        headers = next(reader)
        hd_time = [hd for hd in headers if 'datetime' in hd.lower()]
        hd_icp = [hd for hd in headers if 'icp' in hd.lower()]
    with open(file_path) as csv_file:
        raw_data = pd.read_csv(csv_file, sep=',', usecols=[hd_time[0], hd_icp[0]])

    time = raw_data[hd_time].to_numpy()
    t0 = (time[0] - int(time[0])) * 24 * 3600
    raw_t = (time - np.floor(time)) * 24 * 3600 - t0
    raw_icp = raw_data[hd_icp].to_numpy()

    t = np.delete(raw_t, np.where(np.isnan(raw_icp)))
    icp = np.delete(raw_icp, np.where(np.isnan(raw_icp)))
    fs_hat = round(1/(t[1]-t[0]), -1)

    return t, icp, fs_hat


def detect_icp_pulses(icp_raw, fs=100):
    icp = signal.detrend(icp_raw)
    Wc1 = 20 / fs
    Wc2 = 0.7 / (fs/2)

    b1, a1 = signal.iirfilter(N=8, Wn=Wc1, btype='lowpass', rs=60, rp=1, ftype='cheby1', output='ba')
    f_icp = signal.filtfilt(b1, a1, icp)

    b2, a2 = signal.iirfilter(N=8, Wn=Wc2, btype='highpass', rs=60, rp=1, ftype='cheby1', output='ba')
    f_icp = signal.filtfilt(b2, a2, f_icp)

    dICP = np.diff(f_icp)
    SSF = np.insert(dICP, 0, 0)
    SSF[np.argwhere(SSF < 0)] = 0

    z_icp = np.copy(f_icp)
    z_icp[np.argwhere(z_icp < 0)] = 0
    nSSF = np.multiply(SSF, z_icp)

    min_dist = 0.4 * fs
    pk_locs = signal.find_peaks(nSSF, distance=min_dist)[0]

    po_locs = []
    for pk_ind, pk_loc in enumerate(pk_locs[:-1]):
        w = f_icp[pk_locs[pk_ind]:pk_locs[pk_ind + 1]]
        w_rev = w[::-1]
        offset = np.argmin(w_rev)
        po_locs.append(pk_locs[pk_ind + 1] - offset - 1)

    return f_icp, po_locs

# p = Path(r'test.csv')
# t, icp, fs = read_from_csv_with_datetime(p)
# f_icp, po_locs = detect_icp_pulses(icp, fs)

In [ ]:
import os
import numpy as np
import torch
import datetime as dt
import torch.nn as nn
from torchdiffeq import odeint_adjoint as odeint
is_odenet = True

device = torch.device('cuda:' + str(0) if torch.cuda.is_available() else 'cpu')
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv1d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv1d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


def norm(dim):
    return nn.GroupNorm(min(32, dim), dim)


class ResBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(ResBlock, self).__init__()
        self.norm1 = norm(inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.norm2 = norm(planes)
        self.conv2 = conv3x3(planes, planes)

    def forward(self, x):
        shortcut = x

        out = self.relu(self.norm1(x))

        if self.downsample is not None:
            shortcut = self.downsample(out)

        out = self.conv1(out)
        out = self.norm2(out)
        out = self.relu(out)
        out = self.conv2(out)

        return out + shortcut


class ConcatConv2d(nn.Module):

    def __init__(self, dim_in, dim_out, ksize=3, stride=1, padding=0, dilation=1, groups=1, bias=True, transpose=False):
        super(ConcatConv2d, self).__init__()
        module = nn.ConvTranspose1d if transpose else nn.Conv1d
        self._layer = module(
            dim_in + 1, dim_out, kernel_size=ksize, stride=stride, padding=padding, dilation=dilation, groups=groups,
            bias=bias
        )

    def forward(self, t, x):
        tt = torch.ones_like(x[:, :1, :]) * t
        ttx = torch.cat([tt, x], 1)
        return self._layer(ttx)


class ODEfunc(nn.Module):

    def __init__(self, dim):
        super(ODEfunc, self).__init__()
        self.norm1 = norm(dim)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = ConcatConv2d(dim, dim, 3, 1, 1)
        self.norm2 = norm(dim)
        self.conv2 = ConcatConv2d(dim, dim, 3, 1, 1)
        self.norm3 = norm(dim)
        self.nfe = 0

    def forward(self, t, x):
        self.nfe += 1
        out = self.norm1(x)
        out = self.relu(out)
        out = self.conv1(t, out)
        out = self.norm2(out)
        out = self.relu(out)
        out = self.conv2(t, out)
        out = self.norm3(out)
        return out


class ODEBlock(nn.Module):

    def __init__(self, odefunc):
        super(ODEBlock, self).__init__()
        self.odefunc = odefunc
        self.integration_time = torch.tensor([0, 1]).float()

    def forward(self, x):
        self.integration_time = self.integration_time.type_as(x)
        out = odeint(self.odefunc, x, self.integration_time, rtol=1e-5, atol=1e-5)
        return out[1]

    @property
    def nfe(self):
        return self.odefunc.nfe

    @nfe.setter
    def nfe(self, value):
        self.odefunc.nfe = value


class Flatten(nn.Module):

    def __init__(self):
        super(Flatten, self).__init__()

    def forward(self, x):
        shape = torch.prod(torch.tensor(x.shape[1:])).item()
        return x.view(-1, shape)


downsampling_layers = [
        nn.Conv1d(1, 64, 3, 1),
        ResBlock(64, 64, stride=2, downsample=conv1x1(64, 64, 2)),
        ResBlock(64, 64, stride=2, downsample=conv1x1(64, 64, 2)),
    ]

feature_layers = [ODEBlock(ODEfunc(64))] if is_odenet else [ResBlock(64, 64) for _ in range(6)]
fc_layers = [norm(64), nn.ReLU(inplace=True), nn.AdaptiveAvgPool1d(1), Flatten(), nn.Dropout(0.6), nn.Linear(64, 5)]

model = nn.Sequential(*downsampling_layers, *feature_layers, *fc_layers).to(device)

In [ ]:
from tqdm import tqdm

def get_batches(pulses, batch_size=2048, sig_length=180):
    for i in range(0,len(pulses),batch_size):
        tensors = []
        for pulse in pulses[i:i+batch_size]:
            pulse = np.array(pulse)
            if len(pulse) <= sig_length:
                bckg = np.zeros(sig_length)
                bckg[-len(pulse):] = pulse
            else:
                middle = int(len(pulse)/2)
                bckg = pulse[middle-sig_length/2:middle+sig_lenght/2]
            if len(bckg) != sig_length:
                print(len(bckg))
            tensors.append(bckg)
        yield torch.tensor(tensors,dtype=torch.float)

model_path = os.path.join(os.getcwd(), "experiments", "2020-03-17_ODE_5cls_1", "model_1.pth")

for paths in tqdm(file_paths):
    csv_file = read_from_csv_with_datetime(os.path.join(paths[0], paths[1]))
    f_icp, po_locs = detect_icp_pulses(csv_file[1], csv_file[2])
    pulses = [(f_icp[po_locs[i]: po_locs[i+1]]-min(f_icp[po_locs[i]: po_locs[i+1]]))
                  /(max(f_icp[po_locs[i]: po_locs[i+1]])-min(f_icp[po_locs[i]: po_locs[i+1]])) 
                  for i in range(len(po_locs)-1)]
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu'))['state_dict'])
    model.eval()
    all_outputs = []
    for batch in tqdm(get_batches(pulses)):
        batch = batch.unsqueeze(1)
        outputs = model(batch)
        softmax_outputs = F.softmax(outputs, dim=1).tolist()
        all_outputs += np.argmax(softmax_outputs, axis=1).tolist()
        
    diction = {
    "model_predictions": all_outputs,
    "time": csv_file[0],
    "icp": csv_file[1],
    "fs_hat": csv_file[2],
    "f_icp": f_icp,
    "po_locs": po_locs,
    "pulses": pulses
    }
    with open(os.path.join(os.getcwd(), "results",paths[1].split(".")[0]+".pkl" ), 'wb') as infile:
        loaded = pickle.dump(infile, diction)

In [ ]:
import copy
y = copy.deepcopy(all_outputs)
x = [int((po_locs[i]+ po_locs[i+1])/2) for i in range(len(po_locs)-1)]

In [ ]:
cls1 = [1 if i == 0 else 0 for i in y ]
cls2 = [1 if i == 1 else 0 for i in y ]
cls3 = [1 if i == 2 else 0 for i in y ]
cls4 = [1 if i == 3 else 0 for i in y ]
cls5 = [1 if i == 4 else 0 for i in y ]

In [ ]:
time = [csv_file[0][i] for i in x]

In [ ]:
import json
diction = {
    "model_predictions": y,
    "model_idx": x,
    "model_time": time,
    "time": csv_file[0],
    "icp": csv_file[1],
    "fs_hat": csv_file[2],
    "f_icp": f_icp,
    "po_locs": po_locs,
    "pulses": pulses
}


In [ ]:
import pickle
with open(os.path.join(os.getcwd(), "results",'PAC_1.pkl' ), 'rb') as infile:
    loaded = pickle.load(infile)

In [ ]:
loaded.keys()

In [ ]:
from scipy.signal import blackmanharris
from scipy.stats import pearsonr

def amp_from_fft(sig):
    windowed = sig * blackmanharris(len(sig))
    f = np.fft.fft(windowed)
    amplitudes = 2 / len(sig) * np.abs(f)
    i = np.argmax(amplitudes)
    return amplitudes[i]

time_diff = 300*int(loaded["fs_hat"])
small_time_diff = 10*int(loaded["fs_hat"])
means = []
four = []
for j in range(0, len(loaded["icp"]), small_time_diff):
    means.append(np.mean(loaded["icp"][j:j+small_time_diff]))
    four.append(amp_from_fft(loaded["icp"][j:j+small_time_diff]))
    
raps = []
tms = []
y = loaded['model_predictions']
cls1 = [1 if i == 0 else 0 for i in y ]
cls2 = [1 if i == 1 else 0 for i in y ]
cls3 = [1 if i == 2 else 0 for i in y ]
cls4 = [1 if i == 3 else 0 for i in y ]
cls5 = [1 if i == 4 else 0 for i in y ]

m1=[]
m2=[]
m3=[]
m4=[]
m5=[]
po_locs = loaded["po_locs"]
model_times = [loaded["time"][int(po_locs[i+1])] for i in range(len(po_locs)-1)]
for i in range(30, len(means), 6):
    raps.append(pearsonr(means[i-30:i], four[i-30:i])[0])
    time_start = loaded["time"][(i-30)*small_time_diff]
    time_end = loaded["time"][i*small_time_diff]
    times = [1 if time_start<=t<time_end else 0 for t in model_times]
    m1.append(sum([cls1[i] if times[i]==1 else 0 for i in range(len(cls1))])/sum(times))
    m2.append(sum([cls2[i] if times[i]==1 else 0 for i in range(len(cls2))])/sum(times))
    m3.append(sum([cls3[i] if times[i]==1 else 0 for i in range(len(cls3))])/sum(times))
    m4.append(sum([cls4[i] if times[i]==1 else 0 for i in range(len(cls4))])/sum(times))
    m5.append(sum([cls5[i] if times[i]==1 else 0 for i in range(len(cls5))])/sum(times))
    tms.append(time_end)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=tms, y=raps,
                    mode='lines',
                    name='Rap'))
fig.add_trace(go.Scatter(x=tms, y=m1,
                    mode='lines',
                    name='T1'))
fig.add_trace(go.Scatter(x=tms, y=m2,
                    mode='lines',
                    name='T2'))
fig.add_trace(go.Scatter(x=tms, y=m3,
                    mode='lines',
                    name='T3'))
fig.add_trace(go.Scatter(x=tms, y=m4,
                    mode='lines',
                    name='T4'))
fig.add_trace(go.Scatter(x=tms, y=m5,
                    mode='lines',
                    name='A+E'))
fig.update_layout(title='Changes in Rap value and percentages of predicted classes',
                   xaxis_title='Time[seconds]')

fig.show()

In [ ]:
from scipy.signal import blackmanharris, medfilt, detrend, filtfilt, iirfilter
from scipy.stats import pearsonr
import pickle
import plotly.graph_objects as go
import plotly


def amp_from_fft(sig, fs):
#     windowed = sig * blackmanharris(len(sig))
    f = np.fft.fft(sig)
    freqs = np.fft.fftfreq(len(sig), 1/fs)
    amplitudes = 2 / len(sig) * np.abs(f)
#     print(amplitudes)
#     input()
    amps = [amplitudes[i] if 0.66<=freqs[i]<=3 else 0 for i in range(len(amplitudes)) ]
#     amps=amplitudes[1:]
    return max(amps)

def read_from_csv_with_datetime(file_path):
    with open(file_path) as csv_file:
        reader = csv.reader(csv_file, delimiter=',')
        headers = next(reader)
        hd_time = [hd for hd in headers if 'datetime' in hd.lower()]
        hd_icp = [hd for hd in headers if 'icp' in hd.lower()]
    with open(file_path) as csv_file:
        raw_data = pd.read_csv(csv_file, sep=',', usecols=[hd_time[0], hd_icp[0]])

    time = raw_data[hd_time].to_numpy()
    t0 = (time[0] - int(time[0])) * 24 * 3600
    raw_t = (time - np.floor(time)) * 24 * 3600 - t0
    raw_icp = raw_data[hd_icp].to_numpy()

    t = [i*(raw_t[1]-raw_t[0]) for i in range(len(raw_t))]

    t = np.delete(t, np.where(np.isnan(raw_icp)))
    icp = np.delete(raw_icp, np.where(np.isnan(raw_icp)))
    fs_hat = round(1/(t[1]-t[0]), -1)

    return t, icp, fs_hat

def read_rap(file_path):
    with open(file_path) as csv_file:
        reader = csv.reader(csv_file, delimiter=',')
        headers = next(reader)
        hd_time = [hd for hd in headers if 'datetime' in hd.lower()]
        hd_icp = [hd for hd in headers if 'rap' in hd.lower()]
    with open(file_path) as csv_file:
        raw_data = pd.read_csv(csv_file, sep=',', usecols=[hd_time[0], hd_icp[0]])

    time = raw_data[hd_time].to_numpy()
    
    clear_time = np.where(np.isnan(time))
    time = np.delete(time, clear_time)
#     t0 = (time[0] - np.floor(time[0])) * 24 * 3600
    raw_t = (time - np.floor(time)) * 24 * 3600
    raw_t = raw_t - raw_t[0]
    raw_icp = raw_data[hd_icp].to_numpy()
    raw_icp = np.delete(raw_icp, clear_time)
    t = [i*(raw_t[1]-raw_t[0]) for i in range(len(raw_t))]
    t = np.delete(t, np.where(np.isnan(raw_icp)))
    icp = np.delete(raw_icp, np.where(np.isnan(raw_icp)))

    return t, icp


def files(path, ext=None, recursive=True):
    for (dirpath, dirnames, filenames) in os.walk(path):
        if recursive or dirpath==path:
            for file in filenames:
                if os.path.isfile(os.path.join(dirpath, file)):
                    f_dta = None
                    for file_dta in filenames:
                        if file.split(".")[0] in file_dta and file != file_dta:
                            f_dta = file_dta                
                    if ext is None:
                        yield dirpath, file, f_dta
                    elif ext in file.split(".")[-1]:
                        yield dirpath, file, f_dta

avl_files = [paths for paths in files(os.path.join(os.getcwd(), "results"), "pkl", False)]
signal_files = [paths for paths in files(os.path.join(os.getcwd(), "datasets", "Longer"), "csv")]
rap_files = [paths for paths in files(os.path.join(os.getcwd(), "results", "RAP", "RAP"), "csv")]

@interact(i = (0, len(avl_files)-1, 1),show_signal = (0, 1, 1))
def plot_file(i=82, show_signal=0):
    with open(os.path.join(avl_files[i][0], avl_files[i][1]), 'rb') as infile:
        loaded = pickle.load(infile)
        
    for j in range(len(rap_files)):
            rap_name = rap_files[j][1].split(".")[0].split("_")
            if "r1" in rap_name:
                rap_name = "_".join(rap_name[:-1])
            else:
                rap_name = "_".join(rap_name)
            
            if avl_files[i][1].split(".")[0] == rap_name:
                time, RAP = read_rap(os.path.join(rap_files[j][0], rap_files[j][1]))
                break
        
    y = loaded["all_outputs"]
    tms = loaded["time"]
    raps = loaded["RAP"]
    m1 = loaded["T1"]
    m2 = loaded["T2"]
    m3 = loaded["T3"]
    m4 = loaded["T4"]
    m5 = loaded["A+E"]
    tms = [i * (tms[1]-tms[0]) for i in range(len(raps))]
    if show_signal == 1:
        layout = go.Layout(
        yaxis=dict(
            domain=[0, 0.5]
        ),
        yaxis2=dict(
            domain=[0.5, 1]
        ))
        fig = go.Figure(layout=layout)
    else:
        fig = go.Figure()
#     print(time[-1], tms[-1])
    fig.add_trace(go.Scatter(x=time, y=RAP,
                                mode='lines',
                                name='RAP')) 
#     fig.add_trace(go.Scatter(x=tms, y=raps,
#                         mode='lines',
#                         name='Rap'))
    fig.add_trace(go.Scatter(x=tms, y=m1,
                        mode='lines',
                        name='T1'))
    fig.add_trace(go.Scatter(x=tms, y=m2,
                        mode='lines',
                        name='T2'))
    fig.add_trace(go.Scatter(x=tms, y=m3,
                        mode='lines',
                        name='T3'))
    fig.add_trace(go.Scatter(x=tms, y=m4,
                        mode='lines',
                        name='T4'))
    fig.add_trace(go.Scatter(x=tms, y=m5,
                        mode='lines',
                        name='A+E'))
    if show_signal == 1:
        for j in range(len(signal_files)):
            if avl_files[i][1].split(".")[0] == signal_files[j][1].split(".")[0]:
                time, signal, freq = read_from_csv_with_datetime(os.path.join(signal_files[j][0], signal_files[j][1]))
        step = int(freq * 60)
        time = [i*(time[1]-time[0]) for i in range(len(time))]
        signalc = np.convolve(signal, np.ones((step,))/step, mode='valid')
        fig.add_trace(go.Scatter(x=time[::step], y=signalc[::step],
                        mode='lines',
                        name='ICP', yaxis="y2")) 

    
    fig.update_layout(title='Changes in Rap value and percentages of predicted classes for ' + avl_files[i][1].split('.')[0],
                       xaxis_title='Time[seconds]')

    fig.show()
    plotly.offline.plot(fig, filename=os.path.join(os.getcwd(), "results", "graphs", avl_files[i][1].split('.')[0]+'_bez_okna.html'), auto_open=False)

In [ ]:
import re

def replace(file, pattern, subst):
    # Read contents from file as a single string
    file_handle = open(file, 'r')
    file_string = file_handle.read()
    file_handle.close()

    # Use RE package to allow for replacement (also allowing for (multiline) REGEX)
    file_string = (re.sub(pattern, subst, file_string))

    # Write contents to file.
    # Using mode 'w' truncates the file.
    file_handle = open(file, 'w')
    file_handle.write(file_string)
    file_handle.close()
    
rap_files = [paths for paths in files(os.path.join(os.getcwd(), "results", "RAP", "RAP"), "csv")]
for path in rap_files:
    filename = os.path.join(path[0], path[1])
    replace(filename, "DateTime,RAP,Amp,icp", "DateTime,RAP,Amp,icp,,")

In [ ]:
from scipy.signal import blackmanharris, medfilt, detrend, filtfilt, iirfilter
from scipy.stats import pearsonr
import pickle
import plotly.graph_objects as go
import plotly
from tqdm import tqdm

def amp_from_fft(sig, fs):
#     windowed = sig * blackmanharris(len(sig))
    f = np.fft.fft(sig)
    freqs = np.fft.fftfreq(len(sig), 1/fs)
    amplitudes = 2 / len(sig) * np.abs(f)
#     print(amplitudes)
#     input()
    amps = [amplitudes[i] if 0.66<=freqs[i]<=3 else 0 for i in range(len(amplitudes)) ]
#     amps=amplitudes[1:]
    return max(amps)

def read_rap(file_path):
    with open(file_path) as csv_file:
        reader = csv.reader(csv_file, delimiter=',')
        headers = next(reader)
        hd_time = [hd for hd in headers if 'datetime' in hd.lower()]
        hd_icp = [hd for hd in headers if 'rap' in hd.lower()]
    with open(file_path) as csv_file:
        raw_data = pd.read_csv(csv_file, sep=',', usecols=[hd_time[0], hd_icp[0]])

    time = raw_data[hd_time].to_numpy()
    
    clear_time = np.where(np.isnan(time))
    time = np.delete(time, clear_time)
#     t0 = (time[0] - np.floor(time[0])) * 24 * 3600
    raw_t = (time - np.floor(time)) * 24 * 3600
    raw_t = raw_t - raw_t[0]
    raw_icp = raw_data[hd_icp].to_numpy()
    raw_icp = np.delete(raw_icp, clear_time)
    t = [i*(raw_t[1]-raw_t[0]) for i in range(len(raw_t))]
    t = np.delete(t, np.where(np.isnan(raw_icp)))
    icp = np.delete(raw_icp, np.where(np.isnan(raw_icp)))

    return t, icp


def files(path, ext=None, recursive=True):
    for (dirpath, dirnames, filenames) in os.walk(path):
        if recursive or dirpath==path:
            for file in filenames:
                if os.path.isfile(os.path.join(dirpath, file)):
                    f_dta = None
                    for file_dta in filenames:
                        if file.split(".")[0] in file_dta and file != file_dta:
                            f_dta = file_dta                
                    if ext is None:
                        yield dirpath, file, f_dta
                    elif ext in file.split(".")[-1]:
                        yield dirpath, file, f_dta

def read_from_csv_with_datetime(file_path):
    with open(file_path) as csv_file:
        reader = csv.reader(csv_file, delimiter=',')
        headers = next(reader)
        hd_time = [hd for hd in headers if 'datetime' in hd.lower()]
        hd_icp = [hd for hd in headers if 'icp' in hd.lower()]
    with open(file_path) as csv_file:
        raw_data = pd.read_csv(csv_file, sep=',', usecols=[hd_time[0], hd_icp[0]])

    time = raw_data[hd_time].to_numpy()
    t0 = (time[0] - int(time[0])) * 24 * 3600
    raw_t = (time - np.floor(time)) * 24 * 3600 - t0
    raw_icp = raw_data[hd_icp].to_numpy()

    t = np.delete(raw_t, np.where(np.isnan(raw_icp)))
    icp = np.delete(raw_icp, np.where(np.isnan(raw_icp)))
    fs_hat = round(1/(t[1]-t[0]), -1)

    return t, icp, fs_hat


def detect_icp_pulses(icp_raw, fs=100):
    icp = signal.detrend(icp_raw)
    Wc1 = 20 / fs
    Wc2 = 0.7 / (fs/2)

    b1, a1 = signal.iirfilter(N=8, Wn=Wc1, btype='lowpass', rs=60, rp=1, ftype='cheby1', output='ba')
    f_icp = signal.filtfilt(b1, a1, icp)

    b2, a2 = signal.iirfilter(N=8, Wn=Wc2, btype='highpass', rs=60, rp=1, ftype='cheby1', output='ba')
    f_icp = signal.filtfilt(b2, a2, f_icp)

    dICP = np.diff(f_icp)
    SSF = np.insert(dICP, 0, 0)
    SSF[np.argwhere(SSF < 0)] = 0

    z_icp = np.copy(f_icp)
    z_icp[np.argwhere(z_icp < 0)] = 0
    nSSF = np.multiply(SSF, z_icp)

    min_dist = 0.4 * fs
    pk_locs = signal.find_peaks(nSSF, distance=min_dist)[0]

    po_locs = []
    for pk_ind, pk_loc in enumerate(pk_locs[:-1]):
        w = f_icp[pk_locs[pk_ind]:pk_locs[pk_ind + 1]]
        w_rev = w[::-1]
        offset = np.argmin(w_rev)
        po_locs.append(pk_locs[pk_ind + 1] - offset - 1)

    return f_icp, po_locs

# p = Path(r'test.csv')
# t, icp, fs = read_from_csv_with_datetime(p)
# f_icp, po_locs = detect_icp_pulses(icp, fs)

avl_files = [paths for paths in files(os.path.join(os.getcwd(), "results", "CNN"), "pkl", False)]
signal_files = [paths for paths in files(os.path.join(os.getcwd(), "datasets", "Longer"), "csv")]
rap_files = [paths for paths in files(os.path.join(os.getcwd(), "results", "RAP", "RAP"), "csv")]


# @interact(i = (0, len(avl_files)-1, 1))
def plot_file(i=23):
#     print(avl_files[i][1])
    with open(os.path.join(avl_files[i][0], avl_files[i][1]), 'rb') as infile:
        loaded = pickle.load(infile)
    with open(os.path.join(avl_files[i][0], "times", avl_files[i][1]), 'rb') as infile:
        times = pickle.load(infile)
    
    for j in range(len(rap_files)):
            rap_name = rap_files[j][1].split(".")[0].split("_")
            if "r1" in rap_name:
                rap_name = "_".join(rap_name[:-1])
            else:
                rap_name = "_".join(rap_name)
            
            if avl_files[i][1].split(".")[0] == rap_name:
                time, RAP = read_rap(os.path.join(rap_files[j][0], rap_files[j][1]))
                break
        
    for j in range(len(signal_files)):
            if avl_files[i][1].split(".")[0] == signal_files[j][1].split(".")[0]:
                time_signal, signal, freq = read_from_csv_with_datetime(os.path.join(signal_files[j][0], signal_files[j][1]))
#                 f_icp, po_locs = detect_icp_pulses(signal, freq)
#                 print(len(po_locs))
                break
                
    y = loaded["all_outputs"]
    data_over = []
    data_under = []
    
    model_times = times["model_times"]
    jmp = freq*60
    idx = 0
    for j in range(len(time)):
        time_start = time_signal[idx]
        while(time_signal[idx] < time[j] and idx<=len(time_signal)-2):
            idx+=1
        time_end = time_signal[idx-1]
        data = [y[t] for t in range(len(model_times)) if time_start<=model_times[t]<time_end ]
        if RAP[j] >= 0.6:
            data_over += data
        else:
            data_under += data
                
    data_over += [0, 1, 2, 3, 4]
    data_under += [0, 1, 2, 3, 4]
    
    counts_over = np.unique(data_over, return_counts=True)
    counts_under = np.unique(data_under, return_counts=True)
    
    counts_under = [count - 1 for count in counts_under[1]]
    counts_over = [count - 1 for count in counts_over[1]]
    
    labels = ["T1", "T2", "T3", "T4", "A+E"]
    print(counts_under)
#     layout = go.Layout(
#     yaxis=dict(
#         domain=[0, 0.5]
#     ),
#     yaxis2=dict(
#         domain=[0.5, 1]
#     ))
    fig = go.Figure()
    fig.add_trace(go.Bar(
            x=labels,
            y=counts_over, name="RAP>=0.6"))
    fig.add_trace(go.Bar(x=labels, y=counts_under, name="RAP<0.6"))
    
    fig.update_layout(title='Histogramy dla chwilowego RAP dla przebiegu ' + avl_files[i][1].split('.')[0],
                       xaxis_title='Klasa', yaxis_title='Liczba wykryć')

#     fig.show()
    plotly.offline.plot(fig, filename=os.path.join(os.getcwd(), "results", "graphs", "wszyscyCNN", avl_files[i][1].split('.')[0]+'_Histogram.html'), auto_open=False)
    
for i in tqdm(range(0,len(avl_files))):
    if i != 86:
        plot_file(i)

In [ ]:
signal_files[j][1]

In [ ]:
from scipy.signal import blackmanharris, medfilt, detrend, filtfilt, iirfilter
from scipy.stats import pearsonr
import pickle
import plotly.graph_objects as go
import plotly
from tqdm import tqdm

def read_from_csv_with_datetime(file_path):
    with open(file_path) as csv_file:
        reader = csv.reader(csv_file, delimiter=',')
        headers = next(reader)
        hd_time = [hd for hd in headers if 'datetime' in hd.lower()]
        hd_icp = [hd for hd in headers if 'icp' in hd.lower()]
    with open(file_path) as csv_file:
        raw_data = pd.read_csv(csv_file, sep=',', usecols=[hd_time[0], hd_icp[0]])

    time = raw_data[hd_time].to_numpy()
    t0 = (time[0] - int(time[0])) * 24 * 3600
    raw_t = (time - np.floor(time)) * 24 * 3600 - t0
    raw_icp = raw_data[hd_icp].to_numpy()

    t = [i*(raw_t[1]-raw_t[0]) for i in range(len(raw_t))]

    t = np.delete(t, np.where(np.isnan(raw_icp)))
    icp = np.delete(raw_icp, np.where(np.isnan(raw_icp)))
    fs_hat = round(1/(t[1]-t[0]), -1)

    return t, icp, fs_hat

def read_rap(file_path):
    with open(file_path) as csv_file:
        reader = csv.reader(csv_file, delimiter=',')
        headers = next(reader)
        hd_time = [hd for hd in headers if 'datetime' in hd.lower()]
        hd_icp = [hd for hd in headers if 'rap' in hd.lower()]
    with open(file_path) as csv_file:
        raw_data = pd.read_csv(csv_file, sep=',', usecols=[hd_time[0], hd_icp[0]])

    time = raw_data[hd_time].to_numpy()
    
    clear_time = np.where(np.isnan(time))
    time = np.delete(time, clear_time)
#     t0 = (time[0] - np.floor(time[0])) * 24 * 3600
    raw_t = (time - np.floor(time)) * 24 * 3600
    raw_t = raw_t - raw_t[0]
    raw_icp = raw_data[hd_icp].to_numpy()
    raw_icp = np.delete(raw_icp, clear_time)
    t = [i*(raw_t[1]-raw_t[0]) for i in range(len(raw_t))]
    t = np.delete(t, np.where(np.isnan(raw_icp)))
    icp = np.delete(raw_icp, np.where(np.isnan(raw_icp)))

    return t, icp


def files(path, ext=None, recursive=True):
    for (dirpath, dirnames, filenames) in os.walk(path):
        if recursive or dirpath==path:
            for file in filenames:
                if os.path.isfile(os.path.join(dirpath, file)):
                    f_dta = None
                    for file_dta in filenames:
                        if file.split(".")[0] in file_dta and file != file_dta:
                            f_dta = file_dta                
                    if ext is None:
                        yield dirpath, file, f_dta
                    elif ext in file.split(".")[-1]:
                        yield dirpath, file, f_dta

avl_files = [paths for paths in files(os.path.join(os.getcwd(), "results", "ODE"), "pkl", False)]
signal_files = [paths for paths in files(os.path.join(os.getcwd(), "datasets", "Longer"), "csv")]
rap_files = [paths for paths in files(os.path.join(os.getcwd(), "results", "RAP", "RAP"), "csv")]

full_counts_over = np.array([0, 0, 0, 0, 0])
full_counts_under = np.array([0, 0, 0, 0, 0])
labels = ["T1", "T2", "T3", "T4", "T5"]

for i in tqdm(range(len(avl_files))):
    with open(os.path.join(avl_files[i][0], avl_files[i][1]), 'rb') as infile:
        loaded = pickle.load(infile)
        
    for j in range(len(rap_files)):
            rap_name = rap_files[j][1].split(".")[0].split("_")
            if "r1" in rap_name:
                rap_name = "_".join(rap_name[:-1])
            else:
                rap_name = "_".join(rap_name)
            
            if avl_files[i][1].split(".")[0] == rap_name:
                time, RAP = read_rap(os.path.join(rap_files[j][0], rap_files[j][1]))
                break
        
    y = loaded["all_outputs"]
    RAP_mean = np.mean(RAP)
    counts = np.unique(y, return_counts=True)
    if RAP_mean >= 0.6:
        for j in range(len(counts[0])):
            full_counts_over[counts[0][j]] += counts[1][j]
    else:
        for j in range(len(counts[0])):
            full_counts_under[counts[0][j]] += counts[1][j]

# layout = go.Layout(
#     yaxis=dict(
#         domain=[0, 0.5]
#     ),
#     yaxis2=dict(
#         domain=[0.5, 1]
#     ))
fig = go.Figure()
fig.add_trace(go.Bar(
            x=labels,
            y=full_counts_over/sum(full_counts_over), name="RAP>=0.6"))
fig.add_trace(go.Bar(x=labels, y=full_counts_under/sum(full_counts_under), name="RAP<0.6"))
    
fig.update_layout(title='Histogram',
                       xaxis_title='Class', yaxis_title='Percent of examples')

fig.show()
plotly.offline.plot(fig, filename=os.path.join(os.getcwd(), "results", "graphs","CNN", 'Histogram_2_pelny_procenty.html'), auto_open=False)

In [ ]:
from scipy.signal import blackmanharris, medfilt, detrend, filtfilt, iirfilter
from scipy.stats import pearsonr
import pickle
import plotly.graph_objects as go
import plotly
from plotly.subplots import make_subplots
import pandas as pd
from tqdm import tqdm
import plotly.figure_factory as ff

# hist_data = [x1, x2, x3, x4]

# group_labels = ['Group 1', 'Group 2', 'Group 3', 'Group 4']

# # Create distplot with custom bin_size
# fig = ff.create_distplot(hist_data, group_labels, bin_size=[.1, .25, .5, 1])
# fig.show()
def read_from_csv_with_datetime(file_path):
    with open(file_path) as csv_file:
        reader = csv.reader(csv_file, delimiter=',')
        headers = next(reader)
        hd_time = [hd for hd in headers if 'datetime' in hd.lower()]
        hd_icp = [hd for hd in headers if 'icp' in hd.lower()]
    with open(file_path) as csv_file:
        raw_data = pd.read_csv(csv_file, sep=',', usecols=[hd_time[0], hd_icp[0]])

    time = raw_data[hd_time].to_numpy()
    t0 = (time[0] - int(time[0])) * 24 * 3600
    raw_t = (time - np.floor(time)) * 24 * 3600 - t0
    raw_icp = raw_data[hd_icp].to_numpy()

    t = [i*(raw_t[1]-raw_t[0]) for i in range(len(raw_t))]

    t = np.delete(t, np.where(np.isnan(raw_icp)))
    icp = np.delete(raw_icp, np.where(np.isnan(raw_icp)))
    fs_hat = round(1/(t[1]-t[0]), -1)

    return t, icp, fs_hat

def read_rap(file_path):
    with open(file_path) as csv_file:
        reader = csv.reader(csv_file, delimiter=',')
        headers = next(reader)
        hd_time = [hd for hd in headers if 'datetime' in hd.lower()]
        hd_icp = [hd for hd in headers if 'rap' in hd.lower()]
    with open(file_path) as csv_file:
        raw_data = pd.read_csv(csv_file, sep=',', usecols=[hd_time[0], hd_icp[0]])

    time = raw_data[hd_time].to_numpy()
    
    clear_time = np.where(np.isnan(time))
    time = np.delete(time, clear_time)
#     t0 = (time[0] - np.floor(time[0])) * 24 * 3600
    raw_t = (time - np.floor(time)) * 24 * 3600
    raw_t = raw_t - raw_t[0]
    raw_icp = raw_data[hd_icp].to_numpy()
    raw_icp = np.delete(raw_icp, clear_time)
    t = [i*(raw_t[1]-raw_t[0]) for i in range(len(raw_t))]
    t = np.delete(t, np.where(np.isnan(raw_icp)))
    icp = np.delete(raw_icp, np.where(np.isnan(raw_icp)))

    return t, icp


def files(path, ext=None, recursive=True):
    for (dirpath, dirnames, filenames) in os.walk(path):
        if recursive or dirpath==path:
            for file in filenames:
                if os.path.isfile(os.path.join(dirpath, file)):
                    f_dta = None
                    for file_dta in filenames:
                        if file.split(".")[0] in file_dta and file != file_dta:
                            f_dta = file_dta                
                    if ext is None:
                        yield dirpath, file, f_dta
                    elif ext in file.split(".")[-1]:
                        yield dirpath, file, f_dta

avl_files = [paths for paths in files(os.path.join(os.getcwd(), "results"), "pkl", False)]
signal_files = [paths for paths in files(os.path.join(os.getcwd(), "datasets", "Longer"), "csv")]
rap_files = [paths for paths in files(os.path.join(os.getcwd(), "results", "RAP", "RAP"), "csv")]

full_counts = []
raps = []
labels = ["T1", "T2", "T3", "T4", "A+E"]

fig = go.Figure()
# fig = make_subplots(rows=1, cols=5, specs=[[{}, {}, {}, {}, {}]], shared_xaxes=True,
#                     shared_yaxes=True, vertical_spacing=0.001)

patients = []
for i in tqdm(range(len(avl_files))):
    with open(os.path.join(avl_files[i][0], avl_files[i][1]), 'rb') as infile:
        loaded = pickle.load(infile)
    
    for j in range(len(rap_files)):
            rap_name = rap_files[j][1].split(".")[0].split("_")
            if "r1" in rap_name:
                rap_name = "_".join(rap_name[:-1])
            else:
                rap_name = "_".join(rap_name)
            
            if avl_files[i][1].split(".")[0] == rap_name:
                time, RAP = read_rap(os.path.join(rap_files[j][0], rap_files[j][1]))
                break
        
    
    y = loaded["all_outputs"]
    if len(RAP > 0):
        RAP_mean = np.mean(RAP)
        raps.append(RAP_mean)
        y = y + [0, 1, 2, 3, 4]
        counts = np.unique(y, return_counts=True)
        real_counts = [count - 1 for count in counts[1]]
        percent_counts = real_counts/sum(real_counts)
        full_counts.append(percent_counts)
#         full_counts.append(real_counts)
        patients.append(avl_files[i][1])
    
classes = [["T1", "T2", "T3", "T4", "A+E"] for i in range(len(raps))]
classes = [item for sublist in classes for item in sublist]

patients = [[patient, patient, patient, patient, patient] for patient in patients]
patients = [item for sublist in patients for item in sublist]

raps = [[rap, rap, rap, rap, rap] for rap in raps]
raps = [item for sublist in raps for item in sublist]
total_counts =[[np.sum(item), np.sum(item), np.sum(item), np.sum(item), np.sum(item)] for item in full_counts]
total_counts =[item for sublist in total_counts for item in sublist] 
full_counts = [item for sublist in full_counts for item in sublist]
print(np.max(full_counts))
print(len(raps), len(full_counts), len(classes), len(total_counts), len(patients))
diction = {
    "RAP": raps,
    "size": full_counts,
    "class": classes,
    "sum": total_counts,
    "patients": patients
}
df = pd.DataFrame(diction)


# for class_idx in range(len(labels)):
#     bins = 10
#     x = []
#     bars = []
#     for i in range(bins):
#         rap_min = i/bins
#         rap_max = (i+1)/bins
# #         print(rap_min, rap_max)
# #         print(df[df[(df["RAP"] >= rap_min) & (df["RAP"] < rap_max) & (df["class"] == labels[class_idx])]]["size"])
#         filtered = df[(df["RAP"] >= rap_min) & (df["RAP"] < rap_max)]
#         filtered = filtered[df["class"] == labels[class_idx]]
# #         bars.append(np.sum(filtered["size"]))
#         bars.append(np.sum(filtered["size"])/np.sum(filtered["sum"]))
# #         bars.append(np.sum(df[df[(df["RAP"] >= rap_min) & (df["RAP"] < rap_max) & (df["class"] == labels[class_idx])]]["size"]))
#         x.append((rap_max + rap_min)/2)
#     print(bars)
#     fig.add_trace(go.Bar(
#     y=x,
#     x=bars,
#     name=labels[class_idx],
#     orientation='h',
#     ))
    

fig.add_trace(go.Scatter(x=df["class"], y=df["RAP"], marker=go.scatter.Marker(size=df["size"], sizeref=1/50), mode="markers",
                         customdata=df["patients"],
                        hovertemplate=
                                    "<b>%{x}</b><br>" +
                                    "RAP: %{y:,.2f}<br>" +
                                    "Percent: %{marker.size:.2%}<br>" +
                                    "Patient: %{customdata}"
                                    "<extra></extra>"))
# for class_idx in range(len(labels)):
#     class_count = [cnt[class_idx] for cnt in full_counts]
#     fig.add_trace(go.Scatter(x=df["class"][df["class"]==labels[class_idx]],
#                             y0=df["RAP"][df["class"]==labels[class_idx]],
#                             y=df["size"][df["class"]==labels[class_idx]],
#                             name=labels[class_idx],
#                             meanline_visible=True,
#                            spanmode="hard"),1,class_idx+1)
fig.update_layout(title='Histograms of detected classes according to mean RAP',
                       xaxis_title='Percent of detected examples of class', yaxis_title='Mean RAP')

fig.show()
plotly.offline.plot(fig, filename=os.path.join(os.getcwd(), "results", "graphs", 'Histogram-klasowy_procent.html'), auto_open=False)

In [ ]:
from scipy.signal import blackmanharris, medfilt, detrend, filtfilt, iirfilter
from scipy.stats import pearsonr
from scipy import signal
import pickle
import plotly.graph_objects as go
import plotly
from tqdm import tqdm


def amp_from_fft(sig, fs):
#     windowed = sig * blackmanharris(len(sig))
    f = np.fft.fft(sig)
    freqs = np.fft.fftfreq(len(sig), 1/fs)
    amplitudes = 2 / len(sig) * np.abs(f)
#     print(amplitudes)
#     input()
    amps = [amplitudes[i] if 0.66<=freqs[i]<=3 else 0 for i in range(len(amplitudes)) ]
#     amps=amplitudes[1:]
    return max(amps)

def read_from_csv_with_datetime(file_path):
    with open(file_path) as csv_file:
        reader = csv.reader(csv_file, delimiter=',')
        headers = next(reader)
        hd_time = [hd for hd in headers if 'datetime' in hd.lower()]
        hd_icp = [hd for hd in headers if 'icp' in hd.lower()]
    with open(file_path) as csv_file:
        raw_data = pd.read_csv(csv_file, sep=',', usecols=[hd_time[0], hd_icp[0]])

    time = raw_data[hd_time].to_numpy()
    t0 = (time[0] - int(time[0])) * 24 * 3600
    raw_t = (time - np.floor(time)) * 24 * 3600 - t0
    raw_icp = raw_data[hd_icp].to_numpy()

    t = [i*(raw_t[1]-raw_t[0]) for i in range(len(raw_t))]

    t = np.delete(t, np.where(np.isnan(raw_icp)))
    icp = np.delete(raw_icp, np.where(np.isnan(raw_icp)))
    fs_hat = round(1/(t[1]-t[0]), -1)

    return t, icp, fs_hat

def read_rap(file_path):
    with open(file_path) as csv_file:
        reader = csv.reader(csv_file, delimiter=',')
        headers = next(reader)
        hd_time = [hd for hd in headers if 'datetime' in hd.lower()]
        hd_icp = [hd for hd in headers if 'rap' in hd.lower()]
    with open(file_path) as csv_file:
        raw_data = pd.read_csv(csv_file, sep=',', usecols=[hd_time[0], hd_icp[0]])

    time = raw_data[hd_time].to_numpy()
    
    clear_time = np.where(np.isnan(time))
    time = np.delete(time, clear_time)
#     t0 = (time[0] - np.floor(time[0])) * 24 * 3600
    raw_t = (time - np.floor(time)) * 24 * 3600
    raw_t = raw_t - raw_t[0]
    raw_icp = raw_data[hd_icp].to_numpy()
    raw_icp = np.delete(raw_icp, clear_time)
    t = [i*(raw_t[1]-raw_t[0]) for i in range(len(raw_t))]
    t = np.delete(t, np.where(np.isnan(raw_icp)))
    icp = np.delete(raw_icp, np.where(np.isnan(raw_icp)))

    return t, icp


def files(path, ext=None, recursive=True):
    for (dirpath, dirnames, filenames) in os.walk(path):
        if recursive or dirpath==path:
            for file in filenames:
                if os.path.isfile(os.path.join(dirpath, file)):
                    f_dta = None
                    for file_dta in filenames:
                        if file.split(".")[0] in file_dta and file != file_dta:
                            f_dta = file_dta                
                    if ext is None:
                        yield dirpath, file, f_dta
                    elif ext in file.split(".")[-1]:
                        yield dirpath, file, f_dta
                        
def detect_icp_pulses(icp_raw, fs=100):
    icp = signal.detrend(icp_raw)
    Wc1 = 20 / fs
    Wc2 = 0.7 / (fs/2)

    b1, a1 = signal.iirfilter(N=8, Wn=Wc1, btype='lowpass', rs=60, rp=1, ftype='cheby1', output='ba')
    f_icp = signal.filtfilt(b1, a1, icp)

    b2, a2 = signal.iirfilter(N=8, Wn=Wc2, btype='highpass', rs=60, rp=1, ftype='cheby1', output='ba')
    f_icp = signal.filtfilt(b2, a2, f_icp)

    dICP = np.diff(f_icp)
    SSF = np.insert(dICP, 0, 0)
    SSF[np.argwhere(SSF < 0)] = 0

    z_icp = np.copy(f_icp)
    z_icp[np.argwhere(z_icp < 0)] = 0
    nSSF = np.multiply(SSF, z_icp)

    min_dist = 0.4 * fs
    pk_locs = signal.find_peaks(nSSF, distance=min_dist)[0]

    po_locs = []
    for pk_ind, pk_loc in enumerate(pk_locs[:-1]):
        w = f_icp[pk_locs[pk_ind]:pk_locs[pk_ind + 1]]
        w_rev = w[::-1]
        offset = np.argmin(w_rev)
        po_locs.append(pk_locs[pk_ind + 1] - offset - 1)

    return f_icp, po_locs

avl_files = [paths for paths in files(os.path.join(os.getcwd(), "results"), "pkl", False)]
signal_files = [paths for paths in files(os.path.join(os.getcwd(), "datasets", "Longer"), "csv")]
rap_files = [paths for paths in files(os.path.join(os.getcwd(), "results", "RAP", "RAP"), "csv")]


classes = [[],[],[],[],[]]
patients = [[],[],[],[],[]]
for i in tqdm(range(len(avl_files))):
    with open(os.path.join(avl_files[i][0], "times", avl_files[i][1]), 'rb') as infile:
        times = pickle.load(infile)
    
    with open(os.path.join(avl_files[i][0], avl_files[i][1]), 'rb') as infile:
        loaded = pickle.load(infile)
        
    for j in range(len(rap_files)):
            rap_name = rap_files[j][1].split(".")[0].split("_")
            if "r1" in rap_name:
                rap_name = "_".join(rap_name[:-1])
            else:
                rap_name = "_".join(rap_name)
            
            if avl_files[i][1].split(".")[0] == rap_name:
                time, RAP = read_rap(os.path.join(rap_files[j][0], rap_files[j][1]))
                break
        
    for j in range(len(signal_files)):
            if avl_files[i][1].split(".")[0] == signal_files[j][1].split(".")[0]:
                time_signal, signals, freq = read_from_csv_with_datetime(os.path.join(signal_files[j][0], signal_files[j][1]))
                f_icp, po_locs = detect_icp_pulses(signals, freq)
                break
                
    y = loaded["all_outputs"]
    print(np.unique(y))
    break
    data_over = []
    data_under = []
    model_times = times["model_times"]
    jmp = freq*60
    idx = 0
    for j in range(len(time)):
        time_start = time_signal[idx]
        while(time_signal[idx] < time[j] and idx<=len(time_signal)-2):
            idx+=1
        time_end = time_signal[idx-1]
        data = [y[t] for t in range(len(model_times)-1) if time_start<=model_times[t]<time_end ]
        for d in data:
            classes[d].append(RAP[j])
            patients[d].append("".join(avl_files[i][1].split("_")[:2]))
      


In [ ]:
diction = {
    "classes": classes,
    "patients": patients,
    "labels": ["T1", "T2", "T3", "T4", "A+E"]
    }
with open(os.path.join(os.getcwd(), "results","chwilowe.pkl" ), 'wb') as infile:
     loaded = pickle.dump(diction, infile)

In [ ]:
from scipy.signal import blackmanharris, medfilt, detrend, filtfilt, iirfilter
from scipy.stats import pearsonr
from scipy import signal
import pickle
import plotly.graph_objects as go
import plotly
from tqdm import tqdm

with open(os.path.join(os.getcwd(), "results","chwilowe","chwilowe.pkl" ), 'rb') as infile:
     loaded = pickle.load(infile)
classes = loaded["classes"]
patients = loaded["patients"]
len(classes[4])

In [ ]:
class_dict = {
    0: 0,
    1: 1,
    2: 2,
    3: 3,
    4: 4
}
data_over = []
data_under = []
for i in range(len(classes)):
    for j in range(len(classes[i])):
        if classes[i][j] >= 0.6:
            data_over.append(class_dict[i])
        else:
            data_under.append(class_dict[i])
            
print()

In [ ]:
# counts_over = [class_dict[i] for j in range(len(classes[i])) for i in range(len(classes)) if classes[i,j] >= 0.6]
# counts_under = [class_dict[i] for j in range(len(classes[i])) for i in range(len(classes)) if classes[i,j] < 0.6]
print(counts_over)
counts_over = np.unique(data_over, return_counts=True)
counts_under = np.unique(data_under, return_counts=True)
    
counts_under = [count - 1 for count in counts_under[1]]
counts_over = [count - 1 for count in counts_over[1]]
    
labels = ["T1", "T2", "T3", "T4", "A+E"]

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(
            x=labels,
            y=counts_over, name="RAP>=0.6"))
fig.add_trace(go.Bar(x=labels, y=counts_under, name="RAP<0.6"))
    
fig.update_layout(title='Histogramy dla chwilowego RAP',
                       xaxis_title='Klasa', yaxis_title='Liczba wykryć')

fig.show()
plotly.offline.plot(fig, filename=os.path.join(os.getcwd(), "results", "graphs", 'Chwilowy_Histogram.html'), auto_open=False)
    

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(
            x=labels,
            y=counts_over/sum(counts_over), name="RAP>=0.6"))
fig.add_trace(go.Bar(x=labels, y=counts_under/sum(counts_under), name="RAP<0.6"))
    
fig.update_layout(title='Histogram dla RAP chwilowego',
                       xaxis_title='Klasa', yaxis_title='Procent klas')

fig.show()
plotly.offline.plot(fig, filename=os.path.join(os.getcwd(), "results", "graphs", 'Histogram_2_pelny_procenty.html'), auto_open=False)

In [ ]:
labels = ["T1", "T2", "T3", "T4", "A+E"]
fig = go.Figure()
bars = [[], [], [], [], []]
for class_idx in range(len(classes)):
    bins = 20
    x = []
    for i in range(bins):
        rap_min = i/bins * 2 - 1
        rap_max = (i+1)/bins * 2 - 1
#         print(rap_min, rap_max)
#         print(df[df[(df["RAP"] >= rap_min) & (df["RAP"] < rap_max) & (df["class"] == labels[class_idx])]]["size"])
        bar = [1 for val in classes[class_idx] if rap_min <= val < rap_max]
#         bars.append(np.sum(bar))
        bars[class_idx].append(np.sum(bar))
#         bars.append(np.sum(filtered["size"])/np.sum(filtered["sum"]))
#         bars.append(np.sum(df[df[(df["RAP"] >= rap_min) & (df["RAP"] < rap_max) & (df["class"] == labels[class_idx])]]["size"]))
        x.append((rap_max + rap_min)/2)
#     print(len(bars[class_idx]))

bars = np.array(bars, dtype=np.float32)
for bar_idx in range(len(bars[0])):
#     print(bar_idx)
#     print(bars[:,bar_idx])
    bars[:,bar_idx] = bars[:,bar_idx] / np.sum(bars[:,bar_idx])
#     print(bars[:,bar_idx])
    
for class_idx in range(len(classes)):
    fig.add_trace(go.Bar(
        y=x,
        x=bars[class_idx],
        name=labels[class_idx],
        orientation='h',
    ))
fig.update_layout(title='Histogram klasowy dla RAP chwilowego',
                       xaxis_title='Klasa', yaxis_title='Procent klasy w danym przedziale')

fig.show()
plotly.offline.plot(fig, filename=os.path.join(os.getcwd(), "results", "graphs", 'histogram_klasowy_procent.html'), auto_open=False)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(
            x=labels,
            y=counts_over/sum(counts_over), name="RAP>=0.6"))
fig.add_trace(go.Bar(x=labels, y=counts_under/sum(counts_under), name="RAP<0.6"))
    
fig.update_layout(title='Histogram dla RAP chwilowego',
                       xaxis_title='Klasa', yaxis_title='Procent klas')

fig.show()
plotly.offline.plot(fig, filename=os.path.join(os.getcwd(), "results", "graphs", 'Histogram_2_pelny_procenty.html'), auto_open=False)

In [ ]:
from scipy.signal import blackmanharris, medfilt, detrend, filtfilt, iirfilter
from scipy.stats import pearsonr
import pickle
import plotly.graph_objects as go
import plotly
import re
from tqdm import tqdm

def read_from_csv_with_datetime(file_path):
    with open(file_path) as csv_file:
        reader = csv.reader(csv_file, delimiter=',')
        headers = next(reader)
        hd_time = [hd for hd in headers if 'datetime' in hd.lower()]
        hd_icp = [hd for hd in headers if 'icp' in hd.lower()]
    with open(file_path) as csv_file:
        raw_data = pd.read_csv(csv_file, sep=',', usecols=[hd_time[0], hd_icp[0]])

    time = raw_data[hd_time].to_numpy()
    t0 = (time[0] - int(time[0])) * 24 * 3600
    raw_t = (time - np.floor(time)) * 24 * 3600 - t0
    raw_icp = raw_data[hd_icp].to_numpy()

    t = [i*(raw_t[1]-raw_t[0]) for i in range(len(raw_t))]

    t = np.delete(t, np.where(np.isnan(raw_icp)))
    icp = np.delete(raw_icp, np.where(np.isnan(raw_icp)))
    fs_hat = round(1/(t[1]-t[0]), -1)

    return t, icp, fs_hat

def read_rap(file_path):
    with open(file_path) as csv_file:
        reader = csv.reader(csv_file, delimiter=',')
        headers = next(reader)
        hd_time = [hd for hd in headers if 'datetime' in hd.lower()]
        hd_icp = [hd for hd in headers if 'rap' in hd.lower()]
    with open(file_path) as csv_file:
        raw_data = pd.read_csv(csv_file, sep=',', usecols=[hd_time[0], hd_icp[0]])

    time = raw_data[hd_time].to_numpy()
    
    clear_time = np.where(np.isnan(time))
    time = np.delete(time, clear_time)
#     t0 = (time[0] - np.floor(time[0])) * 24 * 3600
    raw_t = (time - np.floor(time)) * 24 * 3600
    raw_t = raw_t - raw_t[0]
    raw_icp = raw_data[hd_icp].to_numpy()
    raw_icp = np.delete(raw_icp, clear_time)
    t = [i*(raw_t[1]-raw_t[0]) for i in range(len(raw_t))]
    t = np.delete(t, np.where(np.isnan(raw_icp)))
    icp = np.delete(raw_icp, np.where(np.isnan(raw_icp)))

    return t, icp


def files(path, ext=None, recursive=True):
    for (dirpath, dirnames, filenames) in os.walk(path):
        if recursive or dirpath==path:
            for file in filenames:
                if os.path.isfile(os.path.join(dirpath, file)):
                    f_dta = None
                    for file_dta in filenames:
                        if file.split(".")[0] in file_dta and file != file_dta:
                            f_dta = file_dta                
                    if ext is None:
                        yield dirpath, file, f_dta
                    elif ext in file.split(".")[-1]:
                        yield dirpath, file, f_dta

avl_files = [paths for paths in files(os.path.join(os.getcwd(), "results", "ODE"), "pkl", False)]
signal_files = [paths for paths in files(os.path.join(os.getcwd(), "datasets", "Longer"), "csv")]
rap_files = [paths for paths in files(os.path.join(os.getcwd(), "results", "RAP", "RAP"), "csv")]

full_counts_over = np.array([0, 0, 0, 0, 0])
full_counts_under = np.array([0, 0, 0, 0, 0])
labels = ["T1", "T2", "T3", "T4", "T5"]
patient_dictionary = {}
for file in avl_files:
    match = re.search(r"PAC[0-9]*", file[1])
    patient_dictionary[match.group(0)] = {
        "T1": 0,
        "T2": 0,
        "T3": 0,
        "T4": 0,
        "A+E": 0,
        "RAP": [],
        "ICP": []
    }
    
    
for i in tqdm(range(len(avl_files))):
    patient_name = re.search(r"PAC[0-9]*", avl_files[i][1]).group(0)
    RAP = None
    sig = None
    with open(os.path.join(avl_files[i][0], avl_files[i][1]), 'rb') as infile:
        loaded = pickle.load(infile)
        
    for j in range(len(rap_files)):
            rap_name = rap_files[j][1].split(".")[0].split("_")
            if "r1" in rap_name:
                rap_name = "_".join(rap_name[:-1])
            else:
                rap_name = "_".join(rap_name)
            
            if avl_files[i][1].split(".")[0] == rap_name:
                time, RAP = read_rap(os.path.join(rap_files[j][0], rap_files[j][1]))
                break
    
    for j in range(len(signal_files)):
        if avl_files[i][1].split(".")[0] == signal_files[j][1].split(".")[0]:
                time_sig, sig, f = read_from_csv_with_datetime(os.path.join(signal_files[j][0], signal_files[j][1]))
                break
    if RAP is not None and sig is not None:
        patient_dictionary[patient_name]["RAP"] += list(RAP)
        patient_dictionary[patient_name]["ICP"] += list(sig)
    else:
        print("Could not find files for {}. RAP: {}, SIG: {}".format(avl_files[i][1], RAP is not None, sig is not None))
        continue
        
    y = loaded["all_outputs"]
    bins, counts = np.unique(y, return_counts=True)

    for j in range(len(bins)):
        if 0 <= bins[j] <= 3:
            name = "T{}".format(bins[j]+1)
        else:
            name = "A+E"
        patient_dictionary[patient_name][name] += counts[j]
        
    

In [ ]:
from scipy.signal import blackmanharris, medfilt, detrend, filtfilt, iirfilter
from scipy.stats import pearsonr
import pickle
import plotly.graph_objects as go
import plotly
import re
from tqdm import tqdm

def read_from_csv_with_datetime(file_path):
    with open(file_path) as csv_file:
        reader = csv.reader(csv_file, delimiter=',')
        headers = next(reader)
        hd_time = [hd for hd in headers if 'datetime' in hd.lower()]
        hd_icp = [hd for hd in headers if 'icp' in hd.lower()]
    with open(file_path) as csv_file:
        raw_data = pd.read_csv(csv_file, sep=',', usecols=[hd_time[0], hd_icp[0]])

    time = raw_data[hd_time].to_numpy()
    t0 = (time[0] - int(time[0])) * 24 * 3600
    raw_t = (time - np.floor(time)) * 24 * 3600 - t0
    raw_icp = raw_data[hd_icp].to_numpy()

    t = [i*(raw_t[1]-raw_t[0]) for i in range(len(raw_t))]

    t = np.delete(t, np.where(np.isnan(raw_icp)))
    icp = np.delete(raw_icp, np.where(np.isnan(raw_icp)))
    fs_hat = round(1/(t[1]-t[0]), -1)

    return t, icp, fs_hat

def read_rap(file_path):
    with open(file_path) as csv_file:
        reader = csv.reader(csv_file, delimiter=',')
        headers = next(reader)
        hd_time = [hd for hd in headers if 'datetime' in hd.lower()]
        hd_icp = [hd for hd in headers if 'rap' in hd.lower()]
    with open(file_path) as csv_file:
        raw_data = pd.read_csv(csv_file, sep=',', usecols=[hd_time[0], hd_icp[0]])

    time = raw_data[hd_time].to_numpy()
    
    clear_time = np.where(np.isnan(time))
    time = np.delete(time, clear_time)
#     t0 = (time[0] - np.floor(time[0])) * 24 * 3600
    raw_t = (time - np.floor(time)) * 24 * 3600
    raw_t = raw_t - raw_t[0]
    raw_icp = raw_data[hd_icp].to_numpy()
    raw_icp = np.delete(raw_icp, clear_time)
    t = [i*(raw_t[1]-raw_t[0]) for i in range(len(raw_t))]
    t = np.delete(t, np.where(np.isnan(raw_icp)))
    icp = np.delete(raw_icp, np.where(np.isnan(raw_icp)))

    return t, icp


def files(path, ext=None, recursive=True):
    for (dirpath, dirnames, filenames) in os.walk(path):
        if recursive or dirpath==path:
            for file in filenames:
                if os.path.isfile(os.path.join(dirpath, file)):
                    f_dta = None
                    for file_dta in filenames:
                        if file.split(".")[0] in file_dta and file != file_dta:
                            f_dta = file_dta                
                    if ext is None:
                        yield dirpath, file, f_dta
                    elif ext in file.split(".")[-1]:
                        yield dirpath, file, f_dta

avl_files = [paths for paths in files(os.path.join(os.getcwd(), "results", "ODE"), "pkl", False)]
signal_files = [paths for paths in files(os.path.join(os.getcwd(), "datasets", "Longer"), "csv")]
rap_files = [paths for paths in files(os.path.join(os.getcwd(), "results", "RAP", "RAP"), "csv")]

full_counts_over = np.array([0, 0, 0, 0, 0])
full_counts_under = np.array([0, 0, 0, 0, 0])
labels = ["T1", "T2", "T3", "T4", "T5"]
for file in avl_files:
    match = re.search(r"PAC[0-9]*", file[1])
    patient_dictionary[match.group(0)]["A+E"] = 0
    
for i in tqdm(range(len(avl_files))):
    patient_name = re.search(r"PAC[0-9]*", avl_files[i][1]).group(0)
    RAP = None
    sig = None
    with open(os.path.join(avl_files[i][0], avl_files[i][1]), 'rb') as infile:
        loaded = pickle.load(infile)    
    y = loaded["all_outputs"]
    bins, counts = np.unique(y, return_counts=True)

    for j in range(len(bins)):
        if 0 <= bins[j] <= 3:
            name = "T{}".format(bins[j]+1)
        else:
            name = "A+E"
            patient_dictionary[patient_name][name] += counts[j]
        
    

In [ ]:
import pandas as pd
df_dict = {
    "Pac": [],
    "T1": [],
    "T2": [],
    "T3": [],
    "T4": [],
    "A+E": [],
    "RAP_mean": [],
    "RAP_median": [],
    "ICP_mean": [],
    "ICP_median": []
}

for patient in tqdm(patient_dictionary.keys()):
    df_dict["Pac"].append(patient)
    df_dict["T1"].append(patient_dictionary[patient]["T1"])
    df_dict["T2"].append(patient_dictionary[patient]["T2"])
    df_dict["T3"].append(patient_dictionary[patient]["T3"])
    df_dict["T4"].append(patient_dictionary[patient]["T4"])
    df_dict["A+E"].append(patient_dictionary[patient]["A+E"])
    df_dict["RAP_mean"].append(np.mean(patient_dictionary[patient]["RAP"]))
    df_dict["RAP_median"].append(np.median(patient_dictionary[patient]["RAP"]))
    df_dict["ICP_mean"].append(np.mean(patient_dictionary[patient]["ICP"]))
    df_dict["ICP_median"].append(np.median(patient_dictionary[patient]["ICP"]))

df = pd.DataFrame(df_dict).to_csv(os.path.join(os.getcwd(), "results", "CSV", "ODE", "dane.csv"))

    

In [ ]:
patient_dictionary["PAC1"]["A+E"]

In [ ]:
from scipy.signal import blackmanharris, medfilt, detrend, filtfilt, iirfilter
from scipy.stats import pearsonr
import pickle
import plotly.graph_objects as go
import plotly

import re
from tqdm import tqdm
import matplotlib.pyplot as plt

def read_from_csv_with_datetime(file_path):
    with open(file_path) as csv_file:
        reader = csv.reader(csv_file, delimiter=',')
        headers = next(reader)
        hd_time = [hd for hd in headers if 'datetime' in hd.lower()]
        hd_icp = [hd for hd in headers if 'icp' in hd.lower()]
    with open(file_path) as csv_file:
        raw_data = pd.read_csv(csv_file, sep=',', usecols=[hd_time[0], hd_icp[0]])

    time = raw_data[hd_time].to_numpy()
    t0 = (time[0] - int(time[0])) * 24 * 3600
    raw_t = (time - np.floor(time)) * 24 * 3600 - t0
    raw_icp = raw_data[hd_icp].to_numpy()

    t = [i*(raw_t[1]-raw_t[0]) for i in range(len(raw_t))]

    t = np.delete(t, np.where(np.isnan(raw_icp)))
    icp = np.delete(raw_icp, np.where(np.isnan(raw_icp)))
    fs_hat = round(1/(t[1]-t[0]), -1)

    return t, icp, fs_hat

def read_rap(file_path):
    with open(file_path) as csv_file:
        reader = csv.reader(csv_file, delimiter=',')
        headers = next(reader)
        hd_time = [hd for hd in headers if 'datetime' in hd.lower()]
        hd_icp = [hd for hd in headers if 'rap' in hd.lower()]
    with open(file_path) as csv_file:
        raw_data = pd.read_csv(csv_file, sep=',', usecols=[hd_time[0], hd_icp[0]])

    time = raw_data[hd_time].to_numpy()
    
    clear_time = np.where(np.isnan(time))
    time = np.delete(time, clear_time)
#     t0 = (time[0] - np.floor(time[0])) * 24 * 3600
    raw_t = (time - np.floor(time)) * 24 * 3600
    raw_t = raw_t - raw_t[0]
    raw_icp = raw_data[hd_icp].to_numpy()
    raw_icp = np.delete(raw_icp, clear_time)
    t = [i*(raw_t[1]-raw_t[0]) for i in range(len(raw_t))]
    t = np.delete(t, np.where(np.isnan(raw_icp)))
    icp = np.delete(raw_icp, np.where(np.isnan(raw_icp)))

    return t, icp


def files(path, ext=None, recursive=True):
    for (dirpath, dirnames, filenames) in os.walk(path):
        if recursive or dirpath==path:
            for file in filenames:
                if os.path.isfile(os.path.join(dirpath, file)):
                    f_dta = None
                    for file_dta in filenames:
                        if file.split(".")[0] in file_dta and file != file_dta:
                            f_dta = file_dta                
                    if ext is None:
                        yield dirpath, file, f_dta
                    elif ext in file.split(".")[-1]:
                        yield dirpath, file, f_dta

avl_files = [paths for paths in files(os.path.join(os.getcwd(), "results", "ODE"), "pkl", False)]
signal_files = [paths for paths in files(os.path.join(os.getcwd(), "datasets", "Longer"), "csv")]
rap_files = [paths for paths in files(os.path.join(os.getcwd(), "results", "RAP", "RAP"), "csv")]

labels = ["T1", "T2", "T3", "T4", "A+E"]  

    
for i in tqdm(range(68,len(avl_files))):
    with open(os.path.join(avl_files[i][0], "times", avl_files[i][1]), 'rb') as infile:
        times = pickle.load(infile)
    
    with open(os.path.join(avl_files[i][0], avl_files[i][1]), 'rb') as infile:
        loaded = pickle.load(infile)
        
    for j in range(len(signal_files)):
        if avl_files[i][1].split(".")[0] == signal_files[j][1].split(".")[0]:
                time_sig, sig, f = read_from_csv_with_datetime(os.path.join(signal_files[j][0], signal_files[j][1]))
                break
    po_locs = times["po_locs"]
    m_times = times["model_times"]
    y = loaded["all_outputs"]
#     print(np.unique(y))
    shapes = []
    colors = {
        0: "green",
        1: "blue",
        2: "red",
        3: "magenta",
        4: "goldenrod"
    }
#     for j in range(len(po_locs)-1):
#         shapes.append(
#         dict(
#             type="rect",
#             # x-reference is assigned to the x-values
#             xref="x",
#             # y-reference is assigned to the plot paper [0,1]
#             yref="paper",
#             x0=time_sig[po_locs[j]],
#             y0=0,
#             x1=time_sig[po_locs[j+1]],
#             y1=1,
#             fillcolor=colors[y[j]],
#             opacity=0.4,
#             layer="below",
#             line_width=1,
#         ))
    fig = go.Figure()
    po_sigs = [] 
    po_tms = []
    for j in range(len(po_locs)-1):
        middle = (po_locs[j] + po_locs[j+1])/2
        if middle == np.floor(middle):
            po_sigs.append(sig[int(middle)])
            po_tms.append(time_sig[int(middle)])
        else:
            po_sigs.append((sig[int(np.floor(middle))]+sig[int(np.ceil(middle))])/2)
            po_tms.append((time_sig[int(np.floor(middle))]+time_sig[int(np.ceil(middle))])/2)
#         fig.add_trace(go.Scattergl(x=time_sig[po_locs[j]:po_locs[j+1]], y=sig[po_locs[j]:po_locs[j+1]],
#                                     mode='lines',
#                                     name='ICP - '+labels[y[j]],
#                                     line = dict(
#                                     color=colors[y[j]]
#                                     )
#                                     ))
    fig.add_trace(go.Scattergl(x = [time_sig[loc] for loc in po_locs], y = [sig[loc] for loc in po_locs], mode="markers", name='Splits', marker_symbol = "line-ns"
                               ,marker_line_width=2 , marker_color="black"))
    for class_id in range(5):
        x_plot = [po_tms[j] for j in range(len(y)) if y[j] == class_id]
        y_plot = [po_sigs[j] for j in range(len(y)) if y[j] == class_id]
        fig.add_trace(go.Scattergl(x=x_plot, y = y_plot, mode='markers', name=labels[class_id]))
    
    fig.add_trace(go.Scattergl(x=time_sig, y=sig,
                                mode='lines',
                                name='ICP'))
    
#     fig.update_layout(shapes=shapes)
    plotly.offline.plot(fig, filename=os.path.join(os.getcwd(), "results", "graphs","klasyfikacje", avl_files[i][1] + '_przebieg.html'), auto_open=False)
#     break
    
    